In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

# Input decision matrix (alternatives × criteria)
matrix = np.array([
    [0.156, 0.075, 0.099, 0.101],
    [0.081, 0.072, 0.081, 0.093],
    [0.055, 0.086, 0.067, 0.090],
    [0.055, 0.070, 0.080, 0.080],
    [0.067, 0.081, 0.238, 0.071],
    [0.243, 0.057, 0.117, 0.062],
    [0.032, 0.128, 0.017, 0.131],
    [0.023, 0.088, 0.022, 0.134],
    [0.030, 0.119, 0.112, 0.095],
    [0.081, 0.092, 0.125, 0.081],
    [0.011, 0.131, 0.042, 0.063]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10', 'Site11']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

def marcoss(matrix, weights, criteria_types, sites):
    # Step 1: Define AI and AAI
    ai = np.where(np.array(criteria_types) == 'max', 
                  np.max(matrix, axis=0), 
                  np.min(matrix, axis=0))
    
    aai = np.where(np.array(criteria_types) == 'max', 
                   np.min(matrix, axis=0), 
                   np.max(matrix, axis=0))
    
    # Step 2: Extend matrix with AI and AAI
    extended = np.vstack([matrix, ai, aai])

    # Step 3: Normalize extended matrix
    norm_extended = np.zeros_like(extended, dtype=float)
    for j in range(extended.shape[1]):
        if criteria_types[j] == 'max':
            norm_extended[:, j] = extended[:, j] / ai[j]
        else:
            norm_extended[:, j] = aai[j] / extended[:, j]
    
    # Step 4: Apply weights to normalized matrix
    weighted_extended = norm_extended * weights

    # Step 5: Compute S_i, S_AI, S_AAI
    S = weighted_extended.sum(axis=1)
    S_ai, S_aai = S[-2], S[-1]  # Last two rows are AI and AAI
    S_alts = S[:-2]             # Alternatives only

    # Step 6: Compute utility degrees
    K_plus = S_alts / S_ai
    K_minus = S_alts / S_aai

    # Step 7: Final utility function as compromise score
    utility = (K_plus + K_minus) / (
        1 + (1 - K_plus) / K_plus + (1 - K_minus) / K_minus
    )

    # Step 8: Rank alternatives
    ranks = rankdata(-utility, method='min')

    # Step 9: Return results
    return pd.DataFrame({
        'Site': sites,
        'Utility': utility,
        'Rank': ranks
    }).sort_values(by='Rank').reset_index(drop=True)

# Run MARCOS and show result
result_df = marcoss(matrix, weights, criteria_types, sites)
print(result_df)

      Site   Utility  Rank
0    Site5  6.690949     1
1    Site9  2.886109     2
2    Site6  2.804348     3
3   Site10  2.751879     4
4    Site1  2.388201     5
5    Site2  1.389371     6
6   Site11  1.332860     7
7    Site7  1.311560     8
8    Site3  1.270132     9
9    Site4  1.146758    10
10   Site8  0.751587    11
